In [1]:
import ray
from ray import tune
from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.agents.ppo import APPOTrainer
import argparse
from ray.tune.registry import register_env
from pycigar.utils.registry import make_create_env
import yaml
import time
from pycigar.utils.input_parser import input_parser

SAVE_RATE = 2

"""
Load the scenarios configuration file. This file contains the scenario information
for the experiment.
"""
sim_params = input_parser('ieee37busdata')

"""
Register the environment to OpenGymAI. This is necessary, RLlib can find the new environment
with string name env_name_v:version:, ex. SingleRelativeDiscreteCoopEnv_v0.
env_name: name of environment being used.
sim_params: simulation params, it is the scenario configuration.
simulator: the simulator being used, ex. opendss, gridlabd...
tracking_ids: list of ids of devices being tracked during the experiment.
"""

pycigar_params = {"exp_tag": "cooperative_multiagent_ppo",
                  "env_name": "CentralControlPVInverterEnv",
                  "simulator": "opendss",
                  "tracking_ids": ['adversary_inverter_s701a', 'reg1']}
"""
call function make_create_env() to register the new environment to OpenGymAI.
create_env() is a function to create new instance of the environment.
env_name: the registered name of the new environment.
"""
create_env, env_name = make_create_env(pycigar_params, version=0)
register_env(env_name, create_env)
sim_params['scenario_config']['start_end_time'] = [100, 100+750]
del sim_params['attack_randomization']
test_env = create_env(sim_params)
obs_space = test_env.observation_space  # get the observation space, we need this to construct our agent(s) observation input
act_space = test_env.action_space  # get the action space, we need this to construct our agent(s) action output


In [2]:
obs = test_env.reset()
reward = 0
done = False
while not done:
    # for each observation, let the policy decides what to do
    obs, r, done, _ = test_env.step(2)
    reward += r
# plot the result. This will be saved in ./results
#test_env.plot(pycigar_params['exp_tag'], env_name, 0, reward)

In [3]:
from pycigar.utils.logging import logger
Logger = logger()
Logger.log_dict.keys()

dict_keys(['inverter_s701a', 'adversary_inverter_s701a', 'inverter_s701b', 'adversary_inverter_s701b', 'inverter_s701c', 'adversary_inverter_s701c', 'inverter_s712c', 'adversary_inverter_s712c', 'inverter_s713c', 'adversary_inverter_s713c', 'inverter_s714a', 'adversary_inverter_s714a', 'inverter_s714b', 'adversary_inverter_s714b', 'inverter_s718a', 'adversary_inverter_s718a', 'inverter_s720c', 'adversary_inverter_s720c', 'inverter_s722b', 'adversary_inverter_s722b', 'inverter_s722c', 'adversary_inverter_s722c', 'inverter_s724b', 'adversary_inverter_s724b', 'inverter_s725b', 'adversary_inverter_s725b', 'inverter_s727c', 'adversary_inverter_s727c', 'inverter_s728', 'adversary_inverter_s728', 'inverter_s729a', 'adversary_inverter_s729a', 'inverter_s730c', 'adversary_inverter_s730c', 'inverter_s731b', 'adversary_inverter_s731b', 'inverter_s732c', 'adversary_inverter_s732c', 'inverter_s733a', 'adversary_inverter_s733a', 'inverter_s734c', 'adversary_inverter_s734c', 'inverter_s735c', 'advers

In [4]:
import numpy as np
log_dict = Logger.log_dict 

In [5]:
from pycigar.utils.output import pycigar_output_specs
v = pycigar_output_specs(test_env)

In [ ]:
test_env.unwrapped.k.device.get_regulator_device_ids()

In [ ]:
import opendssdirect as dss
directory = '/home/toanngo/Documents/GitHub/ceds-cigar/rl/data/ieee37busdata/ieee37.dss'
dss.run_command('Redirect '+ directory)

In [ ]:
dss.RegControls.AllNames()

In [6]:
len(v['allMeterVoltages']['Min'])
len(log_dict['network']['loss'])

700

In [7]:
len(log_dict['network']['loss'])

700